## <center>Error for delayed SIR using RK4 and estimating from a coarse solution </center>

DONE IN ONE NORM, TWO NORM AND MAX NORM

In [2]:
from math import factorial
import numpy as np

## SIR model with delay
def SIR(beta, gamma):
    def SIR_dde(t, y1, y2):
        S, I, R = y1
        S_d, I_d, R_d = y2
        return np.array([-beta*S*I_d, beta*S*I_d - gamma*I, gamma*I])
    return SIR_dde



## History function
def phi_SIR(t):
    return np.array([0.8, 0.2, 0])   #ASK WHY THIS HISTORY FUNCTION

## Linear interpolation for delay terms
def y_delayed(t, y, t_q):
    j = np.searchsorted(t, t_q)  #Busca el índice donde insertar t_q para mantener el arreglo ordenado.
    j1, j2 = j-1, j     #Toma los dos puntos de la malla que rodean a t_q
    w = (t_q - t[j1]) / (t[j2] - t[j1])      #Aplica interpolación en estas 2 líneas
    y_delayed = (1 - w) * y[j1] + w * y[j2]
    return y_delayed



In [9]:
## RK4 for DDEs
def RK4sys_dde(f, phi, t_0, t_f, tau, N, history=False):

    ## Preliminaries
    h = (t_f - t_0) / N
    t = np.linspace(t_0, t_f, N+1)
    y0 = phi(t_0)
    y = np.zeros((len(t), len(y0)))
    y[0] = y0

    ## EE with linear interpolation for delay term
    for i in range(0, N):
        if t[i] < tau:
            k1 = f(t[i], y[i], phi(t[i] - tau))
            k2 = f(t[i] + 0.5*h, y[i] + h*0.5*k1, phi(t[i] + 0.5*h - tau))
            k3 = f(t[i] + 0.5*h, y[i] + h*0.5*k2, phi(t[i] + 0.5*h - tau))
            k4 = f(t[i] + h, y[i] + h*k3, phi(t[i] + h - tau))
            y[i+1] = y[i] + h*(k1/6 + k2/3 + k3/3 + k4/6)
        else:
            k1 = f(t[i], y[i], y_delayed(t, y, t[i] - tau))
            k2 = f(t[i] + 0.5*h, y[i] + h*0.5*k1, y_delayed(t, y, t[i] + 0.5*h - tau))
            k3 = f(t[i] + 0.5*h, y[i] + h*0.5*k2, y_delayed(t, y, t[i] + 0.5*h - tau))
            k4 = f(t[i] + h, y[i] + h*k3, y_delayed(t, y, t[i] + h - tau))
            y[i+1] = y[i] + h*(k1/6 + k2/3 + k3/3 + k4/6)

    if history == True: ## Wether the solution should include the history (values on the interval [t_0 - tau, t_0])
        t_hist = np.linspace(t_0-tau, t_0, int(N / (t_f // tau) + 1))
        y_hist = np.array([phi(ti) for ti in t_hist])
        t = np.concatenate((t_hist, t))
        y = np.concatenate((y_hist, y))

    return h, t, y


In [11]:
beta = 2
gamma = 1
t_0 = 0
t_f = 10
tau = 5

N=2**7

h, t, y = RK4sys_dde(SIR(beta, gamma), phi_SIR, t_0, t_f, tau, N, False)
h_1 = h ## h step size
y_1 = y[:,-1] ##Numerical solution with h the step size grid U(h)

h, t, y = RK4sys_dde(SIR(beta, gamma), phi_SIR, t_0, t_f, tau, 2*N, False)
h_2 = h ## h/2 step size
y_2 = y[:,-1] ##Numerical solution with h/2 the step size grid

h, t, y = RK4sys_dde(SIR(beta, gamma), phi_SIR, t_0, t_f, tau, 4*N, False)
h_3 = h ## h/4 step size
y_3 = y[:,-1] ##Numerical solution with h/4 the step size grid

##Computing the errors
exact_1 = y_3[0:4*N+1:4]  ##U(h/4)
exact_2 = y_3[0:4*N+1:2]  ##U(h/2)

#Max norm
norm_1 = np.linalg.norm(y_1 - exact_1,np.inf)  ##Tenía puesto exact_1 - y_1 pero lo cambio de orden pq sino sería U(h/2)-U(h)
norm_2 = np.linalg.norm(y_2 - exact_2,np.inf)

#Norm 1
onenorm_1 = h_1*np.linalg.norm(y_1 - exact_1,1)
onenorm_2 = h_2*np.linalg.norm(y_2 - exact_2,1)

#Norm 2
twonorm_1 = np.sqrt(h_1)*np.linalg.norm(y_1 - exact_1)  
twonorm_2 = np.sqrt(h_2)*np.linalg.norm(y_2 - exact_2)

##Estimating the order
order = np.log2(norm_1/norm_2-1)   
print(order)  #order with maximun norm

oneorder = np.log2(onenorm_1/onenorm_2-1)  
print(oneorder)  #Order with norm 1

ordertwo = np.log2(twonorm_1/twonorm_2-1)  
print(ordertwo)  #Order with norm 2






2.000258513504862
2.010584452485376
2.002438261014819
